# Notebook 5 - Introduction to word embeddings 

So far we understand how do classifiers work, so now we are going to see how to develop effective documents representations using features. We will cover:
- one-hot vectors
- term-document and term-term matrices
- TF-IDF and PPMI
- word embeddings
- word2vec

---

Text is only a method of transcribing our thoughts - we use it to communicate with other people... and computers! However, if we give a computer raw textual data it has no idea how to interpret this set of characters (actually sequence of bits). Text written in English makes sense to us because we know how to interpret it: we look for features we were taught are meaningful (separate words, multiple words, idioms), recall in our mind what is their meaning and join them together to get the full author's notion. 

If we want a computer to "understand" the text, we first need to tell it the set of features it should look for. Since we want it to understand the text as we do, we can treat words and their combinations as features having specific meanings. Then, we can try to express the **meaning of each word** in a document. As documents consist of words, we will be able to represent the document meaning as a set of meaningful words.

We have already used words as features once - using one-hot vectors. However, as you remember, a one-hot vector tells us only that the word exists somewhere in the document. It does not express the meaning of a single word. One-hot vectors let us see that in sports articles there will be more sports-related vocabulary ("football", "match", "win", etc.) and in politics articles, there will be more politics-related vocabulary ("election", "minister", "parliament", etc.), so we can classify based on the occurrence of the words only. However, all one-hot-encoded words are independent of each other. To understand it better, let's see a simple example: 

Let's encode "dog, cat, fridge" using one-hot vectors.
- dog [1,0,0]
- cat [0,1,0]
- fridge [0,0,1]
  
Every word is related to a specific term in the vector and there is no situation where a word has more than one non-zero term. If you plot all 3 vectors in the 3d space, they will be perpendicular - meaning independence. What we would like to do, is to capture word meaning similarities - the word "dog" is more similar to the word "cat" than to the word "fridge".

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
fig = plt.figure()
ax = plt.axes(projection='3d')

ax.quiver(0,0,0,1,0,0, length=1)
ax.quiver(0,0,0,0,1,0, length=1)
ax.quiver(0,0,0,0,0,1, length=1)

ax.set_xlim([-0.1, 1])
ax.set_ylim([-0.1, 1])
ax.set_zlim([-0.1, 1])
ax.set_xlabel('"dog" x axis')
ax.set_ylabel('"cat" y axis')
ax.set_zlabel('"fridge" z axis')
ax.text(1,0, 0.1, "dog", color="red")
ax.text(0,1, 0.1, "cat", color="red")
ax.text(0.1,0, 1, "fridge", color="red")
ax.set_title("what we have")
plt.show()

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
fig = plt.figure()
ax = plt.axes(projection='3d')

ax.quiver(0,0,0,0.6,0.4,0, length=1)
ax.quiver(0,0,0,0.4,0.6,0, length=1)
ax.quiver(0,0,0,0,0,1, length=1)

ax.set_xlim([-0.1, 1])
ax.set_ylim([-0.1, 1])
ax.set_zlim([-0.1, 1])
ax.set_xlabel('"dog" x axis')
ax.set_ylabel('"cat" y axis')
ax.set_zlabel('"fridge" z axis')
ax.text(0.6,0.4, 0.1, "dog", color="red")
ax.text(0.4,0.6, 0.1, "cat", color="red")
ax.text(0.1,0, 1, "fridge", color="red")
ax.set_title("what we WANT TO have")
plt.show()

Firstly, let's see some methods for giving words a meaning using sparse vectors.

## 10. Vectorizers

The standard representation of a word meaning in NLP is by using a vector. We can also represent whole documents using vectors.

### 10.1. Classical one-hot vectors

We have already seen them in previous notebooks. One-hot vector can be used to represent a document from the corpus based on words the document contains.
Let's say we have a corpus C of 3 documents d<sub>1</sub>, d<sub>2</sub>, d<sub>3</sub>.

| document number | document content |
|-----------------|------------------|
| d<sub>1</sub>   | "i like apples"  |
| d<sub>2</sub>   | "we like dogs"   |
| d<sub>3</sub>   | "we and i and dogs" |

Now, we can associate each word with a term in a vector: ["i", "like", "apples", "we", "dogs", "and"], and represent documents using vectors (using max pooling).

| document number | document content |
|-----------------|------------------|
| d<sub>1</sub>   | [1, 1, 1, 0, 0, 0]  |
| d<sub>2</sub>   | [0, 1, 0, 1, 1, 0]  |
| d<sub>3</sub>   | [1, 0, 0, 1, 1, 1]  |

Ok, done! But how can we learn the meanings of these words? Let's build the term-document matrix!

### 10.2. Term-document matrix

The term-document matrix is the representation of word occurrences in documents from the corpus. It shows how many times each word occurs in each document. Let's visualize it using our corpus C.

|          | d<sub>1</sub> | d<sub>2</sub> | d<sub>3</sub> |
|----------|---|---|---|
| "i"      | 1 | 0 | 1 |
| "like"   | 1 | 1 | 0 |
| "apples" | 1 | 0 | 0 |
| "we"     | 0 | 1 | 1 |
| "dogs"   | 0 | 1 | 1 |
| "and"    | 0 | 0 | 2 |

If you look at columns you can read vectors representing each document. For example the document d<sub>3</sub> is represented by vector [1,0,0,1,1,2]. 

Now, let's look at another, more advanced term-document matrix for a corpus of 4 Shakespeare plays and 4 selected words from them. Note what is the relation in word occurrences between comedies (As You Like It and Twelfth Night) and other plays (Julius Caesar and Henry V).

<div style="text-align:center"><img src="https://raw.githubusercontent.com/TheRootOf3/ucl-nlp-notebook-series/main/Notebook5/nb_resources/pic1.png" alt="" width="800"/></div>

#### Documents similarity

We can use these counts to determine whether the two documents are similar. The intuitive approach is that **two documents are similar if they have similar vectors** - in other words, they contain similar words.

Again, we can represent each document using word counts creating a 4d vector. Since it's impossible to display a 4-dimensional plot, let's see how we can compare these documents using only counts of words "battle" and "fool" using 2d vectors.

<div style="text-align:center"><img src="https://raw.githubusercontent.com/TheRootOf3/ucl-nlp-notebook-series/main/Notebook5/nb_resources/pic2.png" alt="" width="600"/></div>

As you can see, these two words are discriminating documents very well! As expected, in comedies there will be a higher number of "fools" and barely visible "battles", hence based on the words "battle" and "fool", documents "As You Like It" and "Twelfth Night" are similar. The same applies to "Henry V" and "Julius Caesar", where the number of "battles" is higher than the number of "fools". Of course, in normal implementation, we would use vectors of length |V| to represent a document.


#### Words similarity

However, what is interesting, we can also read **row vectors**! We can use them to represent the **meaning of each word**.

<div style="text-align:center"><img src="https://raw.githubusercontent.com/TheRootOf3/ucl-nlp-notebook-series/main/Notebook5/nb_resources/pic3.png" alt="" width="800"/></div>

For example, we can express the word "fool" using a vector [36, 58, 1, 4] and the word "wit" using another vector [20, 15, 2, 3]. Here we can apply the same intuition as to documents: **Two words are similar when they have similar vectors** - they tend to occur in similar documents. So in theory, word "fool" [36, 58, 1, 4] should be more similar to "wit" [20, 15, 2, 3] than to "battle" [1, 0, 7, 13] and indeed, it is true!

### 10.3. Term-term matrix

Another way to represent **word meanings** is to use the **term-term matrix**. In this matrix, both rows and columns are labeled with words from the corpus vocabulary. To construct a matrix, for each word *w* we look for the context of this word. We can define the context as *k* previous and *k* next words (usually *k* is small ~5). 

For example, if there is a sentence "At the university I work on data science. My computer performance is quite low." and we look for a context +/- 4 words of the word "computer", we will get words: "on", "data", "science", "My", "performance", "is", "quite", "low". Now, every pair of the word *w* and each of these context words gets +1 in the term-term matrix. If we take every word from V look for its context, and note counts of each co-occurrence, we will develop this word-word co-occurrence matrix. 

The intuition behind this is that if two words co-occur (are nearby in text), their meaning is similar. Let's look at the subset of the term-term matrix for the Wikipedia corpus.

<div style="text-align:center"><img src="https://raw.githubusercontent.com/TheRootOf3/ucl-nlp-notebook-series/main/Notebook5/nb_resources/pic4.png" alt="" width="800"/></div>

As you can see, the word "digital" co-occurs more often with "computer" (1670) than with "sugar" (4). On the other hand, "cherry" is much often seen together with "pie" (442) than with "computer" (2). In other words, "cherry" is more similar to "pie" than to "computer", which makes sense.

### 10.4. TF-IDF

Both term-document and term-term matrices are based on the frequencies of words. Because of this, very frequent words like "the" or "he", do not discriminate well because they are too general. Hence, we need a different method, which will punish too frequent words and discriminate well. Let's introduce one based on the term-document matrix.

TF-IDF, Term Frequency - Inverse Document Frequency is of the same format as term-document matrix, but values are calculated in two steps:

**Term Frequency** of a term *t* in a document *d* is simply a logarithm of the number term *t* occurs in *d*: 

<div style="text-align:center"><img src="https://raw.githubusercontent.com/TheRootOf3/ucl-nlp-notebook-series/main/Notebook5/nb_resources/eq2.png" alt="" width="300"/></div>

We use a logarithm to "punish" words which occur very frequently. The number 1 inside the logarithm is added since if there are no occurrences of a word, we would take a logarithm of 0, which is undefined.

The next part of the TF-IDF is called **Inverse Document Frequency** and emphasizes words, that are rare (may discriminate well). Document frequency for a term *t* is simply the number of documents, in which *t* occurs. Since we are intrested in its opposite (we need rare words!), IDF is calculated as:

<div style="text-align:center"><img src="https://raw.githubusercontent.com/TheRootOf3/ucl-nlp-notebook-series/main/Notebook5/nb_resources/eq3.png" alt="" width="200"/></div>

The complete TF-IDF weighted value for a term *t* and a document *d* is given by the product of both TF and IDF:

<div style="text-align:center"><img src="https://raw.githubusercontent.com/TheRootOf3/ucl-nlp-notebook-series/main/Notebook5/nb_resources/eq4.png" alt="" width="200"/></div>

#### TF-IDF worked example

Let's say we have two documents in our corpus:
- d<sub>1</sub> - I like dogs and cats and birds.
- d<sub>2</sub> - I like dogs. I like cats. I like horses. I like dogs, cats, and horses.

We will calculate TF-IDF values for words "like" and "horses".

Step 1 - Calculate TF

Step 1.1 - Term Count for each document

| terms | d<sub>1</sub> | d<sub>2</sub> |         
|-------|---------------|---------------|        
|I      | 1    | 4 |        
|like   | 1    | 4 | 
|and    | 2    | 1 |
|dogs   | 1    | 2 |
|cats   | 1    | 2 |
|birds  | 1    | 0 |
|horses | 0    | 1 |

Step 1.2 - Total number of tokens in documents
- d<sub>1</sub> - 7
- d<sub>2</sub> - 15

Step 1.3 - calculate TF. tf(t, d) = log<sub>10</sub>(1 + number of term t occurrences in document d)

TF for "like"
- d<sub>1</sub> - tf("like", d<sub>1</sub>) = log<sub>10</sub>(1+1) &asymp; 0.3
- d<sub>2</sub> - tf("like", d<sub>2</sub>) = log<sub>10</sub>(4+1) &asymp; 0.7

TF for "horses"
- d<sub>1</sub> - tf("horses", d<sub>1</sub>) = log<sub>10</sub>(0+1) = 0
- d<sub>2</sub> - tf("horses", d<sub>2</sub>) = log<sub>10</sub>(1+1) &asymp; 0.3


Ok, so we have our TF! Now we need to complete step 2 - IDF

Step 2.1 - calculate IDF. IDF(t) = log(total number of documents/number of documents with term t in it)
- IDF("like") = log<sub>10</sub>(2/2) = 0
- IDF("horses") = log<sub>10</sub>(2/1) &asymp; 0.3
  
Step 3 - calculate TF-IDF
TF-IDF for "like"
- **tfidf("like", d<sub>1</sub>) = tf("like", d<sub>1</sub>) * IDF("like") = 0.3 * 0 = 0**
- **tfidf("like", d<sub>2</sub>) = tf("like", d<sub>2</sub>) * IDF("like") = 0.7 * 0 = 0**

As you can see, the TF-IDF for the word "like" is 0, which indicates that this word does not discriminate well. Indeed, it doesn't since it occurs in both documents! Let's see what is the total tf-idf value for "horses".

TF-IDF for "horses"
- **tfidf("horses", d<sub>1</sub>) = tf("horses", d<sub>1</sub>) * IDF("horses") = 0 * 0.3 = 0**
- **tfidf("horses", d<sub>2</sub>) = tf("horses", d<sub>2</sub>) * IDF("horses") = 0.3 * 0.3 = 0.09**

As you can see, the word "horses" has been awarded non-zero value for the document d<sub>2</sub> and its vector clearly show us that it could be used to discriminate at least these two documents.

Let's see how to use the TF-IDF vectorizer from the `scikit-learn` package on the already seen BBC news dataset. 

In [ ]:
import pandas as pd

dataset_file = "https://raw.githubusercontent.com/TheRootOf3/ucl-nlp-notebook-series/main/Notebook5/datasets/bbc-text.csv"

''' uncomment if you want to run it locally '''
# dataset_file = "./datasets/bbc-text.csv"

bbc_data = pd.read_csv(dataset_file)
bbc_data['text'].head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(bbc_data['text'])
tfidf_data = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names()).T

Now, let's inspect how this dataframe looks like. You will see that rows represent words and columns documents - exactly as in the term-document matrix.

In [ ]:
tfidf_data

To inspect the highest values for a certain document (e.g. document 10) we can do the following:

In [ ]:
tfidf_data[10].sort_values(axis=0, ascending=False).head(10)

## 11. Word embeddings

All previous examples used long and sparse vectors of length |V| or |D|. For big corpora, the vocabulary may oscillate around 40,000 (Brown Corpus) or even 13,000,000 (Google N-grams). This is not only a waste of space but also such long vectors are often not discriminating well and require a very long learning process. What's even more problematic is that previous examples either don't give us the word meaning at all (one-hot vectors) or give us only the word meaning related to occurrences in some documents. Since we want the computer to understand text in a similar way that we do, we need another way of words meaning representation. As stated earlier, we want to capture word similarities effectively, also considering contexts in which they are used. 

Hence, we introduce the concept of **embeddings**, dense vectors of length up to 1,000 (however often much shorter). Let's exemplify the idea behind them.

Imagine a small vocabulary containing 5 words: king, queen, man, woman, and princess. The one-hot vector for the queen would look like the one below.

<div style="text-align:center"><img src="https://raw.githubusercontent.com/TheRootOf3/ucl-nlp-notebook-series/main/Notebook5/nb_resources/embed_pic0.jpeg" alt="" width="400"/></div>

We could try to develop a term-document or term-term matrix but this is not how embeddings work. It's more about extracting significant attributes characterizing specific words and contexts in which they are used.

That words are rich entities with many layers of connotation and meaning. Let’s hand-craft some semantic features for these 5 words. We will represent each word as having some value between 0 and 1 for 5 semantic qualities: royalty, masculinity, femininity, age, and edibility.

<div style="text-align:center"><img src="https://raw.githubusercontent.com/TheRootOf3/ucl-nlp-notebook-series/main/Notebook5/nb_resources/embed_pic1.jpeg" alt="" width="600"/></div>

Given the word “king”, it has a high value for the feature “royalty” (because a king is a male member of the royal family) but a low value for femininity (because he is male) and an even lower value for edibility (because we do not normally eat kings). In the above made-up toy dataset, there are 5 semantic features, and we can plot three of these at a time as a 3D scatter plot with each feature being an axis/dimension.

<div style="text-align:center"><img src="https://raw.githubusercontent.com/TheRootOf3/ucl-nlp-notebook-series/main/Notebook5/nb_resources/embed_pic2.jpeg" alt="" width="300"/></div>

You do not have to create word embeddings yourself. Pre-trained word embeddings can be downloaded and used in your models. We will also look into the most popular algorithms for learning static word embeddings from a corpus.

### 11.1. Static vs contextual

Embeddings can be generally divided into two groups. **Static embeddings** learn a single embedding for each word, independently of the context in which that word is used. This method is simpler and faster and gives decent results. On the other hand, **Contextual embeddings** are more complicated and give better performance, since in this case each word may be represented by more than one embedding (if it occurs in many different contexts). Contextual embeddings, as a relatively new concept, take advantage of complex neural networks and are generally harder to learn and develop. However, contextual embeddings are out of the scope of these notebooks.

### 11.2. word2vec

Word2vec is one of the most popular algorithms for learning static embeddings. The intuition of word2vec is that instead of calculating frequencies of words co-occurrences (how often a word *w* occurs near a context word *c*?), it trains a classifier to make a decision: Is word *w* likely to occur near a context word *c*? After training the classifier, the word *w* is represented by the vector of classifier **weights**.

What is interesting, to train the classifier we don't need any additional labels because we can use word co-occurrences as labels! The overview of this algorithm is as follows:
1. Treat the target word and a neighboring context word as positive examples.
2. Randomly sample other words in the lexicon to get negative samples.
3. Use logistic regression to train a classifier to distinguish those two cases.
4. Use the learned weights as the embeddings.

As you can see, we will generate a training set containing both real word co-occurrences from the source (positive samples) and we will randomly generate some artificial word co-occurrences (negative samples).

However, how do we define *a context* for a target word *w*? There are two approaches:
- CBOW - continuous bag-of-words
- SG - skip-gram

**CBOW** makes a window around the target word *w* of *k* words and treats it as a context. In this case, the word order doesn't matter. CBOW is generally faster to train than SG. CBOW tries to predict the target word *w* based on context words. During the learning process, context words are used as an input and the target word is used as an output. 

**SG** also creates a window of surrounding words but in this case their order and distance from the target word *w* matter. The Skip-gram approach works better for infrequent words. The popular version of SG is **skip-gram with negative sampling** (**SGNS**), where the great majority of all training samples are negative. In terms of predicting, skip-gram does it differently than CBOW. SG given the target word tries to predict context words in an appropriate order. 

<div style="text-align:center"><img src="https://raw.githubusercontent.com/TheRootOf3/ucl-nlp-notebook-series/main/Notebook5/nb_resources/cbow_vs_sg.png" alt="" width="600"/></div>

We will use word2vec python package called `Gensim`. You can download pre-trained word embeddings using Gensim as it's specified [here](https://radimrehurek.com/gensim/auto_examples/howtos/run_downloader_api.html) (we will also do it later).

### 11.3. GloVe

GloVe - Global Vectors - is another model for word embeddings. It is similar to word2vec algorithms in terms of the unsupervised learning approach but it differs when it comes to learning weights. The GloVe is based on the term-term(context) co-occurrence matrix, where it tries to learn the words co-occurrence probability ratios. The fundamental principle behind GloVe is that word similarities can be captured using ratios of conditional probabilities. Let's see an example.

<div style="text-align:center"><img src="https://raw.githubusercontent.com/TheRootOf3/ucl-nlp-notebook-series/main/Notebook5/nb_resources/glove.png" alt="" width="600"/></div>

We know that the word "solid" is more similar to "ice" than to "steam" but if we want to represent this fact, we need to relate both information. As you can see, if we take the ratio of both probabilities, it clearly shows, that the words "ice" and "steam" differ in terms of the context word "solid". Thus, the word "solid" is important since it captures the difference in similarity between these two words. Now, what happens with the word "fashion"? As you can see, from raw probabilities P("fashion"|"ice") and P("fashion"|"steam") we cannot infer the lack of the relationship. When we divide both numbers, we can see that the result is almost 1 (meaning the lack of relationship). According to GloVe researchers - "Compared to the raw probabilities, the ratio is better able to distinguish relevant words (solid and gas) from irrelevant words (water and fashion) and it is also better able to discriminate between the two relevant words.". The GloVe uses a different technique to extract word embedding weights. It is based on word-context matrix factorization and extracting weights directly from this matrix.

You can download glove vectors from the original GloVe website [here](https://nlp.stanford.edu/projects/glove/)! I suggest reading [this article](https://medium.com/analytics-vidhya/basics-of-using-pre-trained-glove-vectors-in-python-d38905f356db) for your first steps with GloVe.

### 11.4. Embeddings example with word2vec
There are two ways we can take advantage of word2vec:
- we can use it to create our own word embedding model
- we can use pre-trained word embeddings models 
  
Let's start with the first approach.

#### Training the model
We will need some data to train the model. Let's use the BBC news articles dataset again.

In [ ]:
!pip install gensim==4.0.1

In [ ]:
import gensim
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
bbc_data['text'].head()

As usual, we can't feed the algorithm with raw data, we need to preprocess them.

In [ ]:
def preprocess_doc(doc):
    # Tokenize
    split_doc = word_tokenize(doc)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    cleaned_doc = []
    for word in split_doc:
        if word not in stop_words:
            cleaned_doc.append(word)
    return cleaned_doc

In [ ]:
cleaned_docs = []
for doc in bbc_data["text"]:
    cleaned_docs.append(preprocess_doc(doc))

Now, we have a list of tokenized documents. We can do one more thing before training the model - extract phrases. We can use Gensim Phrases to replace common words co-occurrences with a single token (e.g. "Mr" + "Johnson" = "Mr_Johnson"). 

In [ ]:
from gensim.models.phrases import Phraser, Phrases

# develop phrases
phrases = Phrases(cleaned_docs)

# Let's see some examples
list(phrases.vocab)[:10]

In [ ]:
# replace current word co-occurrences with phrases
bigrams = Phraser(phrases)
cleaned_docs_phrases = bigrams[cleaned_docs]

Ok, now we can train our model using preprocessed data. Word2Vec model has many parameters, which you can find and explore in the Gensim Word2Vec [documentation](https://radimrehurek.com/gensim/models/word2vec.html). Let's cover the most important parameters:
- `min_count` - if a word occurrence count in the corpus is lower than this value it will be ignored (helpful, since ignores accidental words with typos).
- `window` - this specifies what is the size of the context window for CBOW and skip-gram
- `vector_size` - this number specifies the length of the embedding vector
- `workers` - number of machine cores used for training (speed up drastically the learning process)

In [ ]:
# Initialize the model. Change workers paramater to 1 if your machine does not support multicore processing.
model = gensim.models.Word2Vec(min_count=5, window=4, workers=4, vector_size=100)

# Build vocabulary
model.build_vocab(cleaned_docs_phrases)
# Train the model. total_examples is the size of the vocabulary
model.train(cleaned_docs_phrases, total_examples=model.corpus_count, epochs=30)

Here are the learned vectors!

In [ ]:
model.wv.vectors

If you want to get a vector for a specific word, you can use the `.get_vector()` method or simply index the wv object:

In [ ]:
model.wv.get_vector("football")

or

In [ ]:
model.wv["football"]

#### We can also use pre-trained models 

We often don't need to train our model, since there is a lot of existing and well pre-trained models. Let's see how to use pre-trained word embeddings using word2vec from the `gensim` package. Firstly, we need to **download a pre-trained word embeddings model**. We will use the model trained on google news containing about 100 billion words. 

**Note: It weights around 1.5GB!**

In [ ]:
import gensim.downloader as api
google_wv = api.load('word2vec-google-news-300')

In [ ]:
# word2vec-google-news-300 model embedding vector length is 300
google_wv.get_vector("dog")

Ok, but what do we do with these vectors? How can we use them? Let's explore it in the next section.

### 11.5. Embeddings application - similarity measurement

Word embeddings are used as weights in neural networks, but since NNs are out of the scope of this article, we will use them for something different. The other application of embeddings is the **word and document similarity measurement**. So far, we have already developed word embeddings and we understand that two words are similar if their vectors point in a "similar" direction and are opposite otherwise. But what does the "similar" direction actually mean? We need to develop a mathematical way to measure this similarity. 

#### Cosine similarity measure

We need to define a **measure of vector similarity**, that takes two vectors as an input and returns a measure of their similarity. The most common similarity metric is the **cosine of the angle between two vectors**. Intuitively, if the angle between two vectors is small (cosine is large) then these two vectors point in a similar direction. If the angle between two vectors is large (cosine is small) then these vectors point in different (or even opposite) directions.

The cosine of two vectors *v* and *w* is given by the following equation:

<div style="text-align:center"><img src="https://raw.githubusercontent.com/TheRootOf3/ucl-nlp-notebook-series/main/Notebook5/nb_resources/eq1.png" alt="" width="400"/></div>

where |v| means the length of the vector *v*. However, why do we divide by these lengths? Vectors differ not only in direction but also in length. Generally, since vector terms represent word counts in documents (term-document matrix) or word co-occurrences (term-term matrix), frequent words will have much longer vectors than rare words. Thus, we need to **normalize** their lengths so they don't affect the measure - we want to know how much two vectors differ regardless of their lengths. 

Having all this knowledge, we can try to check the cosine measure between some word embeddings from our model.

Let's manually check the relationship between words "ball", "match" and "president". Bear in mind that cosine values are from -1 to 1 (-1 totally opposite, 1 perfectly similar). In the following example, we are going to use our **BBC model**.

In [ ]:
ball_embedding = model.wv.get_vector("ball")
match_embedding = model.wv.get_vector("match")
president_embedding = model.wv.get_vector("president")

In [ ]:
from numpy import dot
from numpy.linalg import norm

def vector_cos(v1, v2):
    return dot(v1, v2)/(norm(v1) * norm(v2))

In [ ]:
print(vector_cos(ball_embedding, match_embedding))
print(vector_cos(ball_embedding, president_embedding))
print(vector_cos(match_embedding, president_embedding))

As expected, the words "ball" and "match" tend to have a similar meaning, while the word "president" shows the difference in meaning with both "ball" and "match". Of course, we don't have to manually make all these calculations. We can use built-in methods from Gensim Word2Vec.

In [ ]:
# For example, we may want to find most similar words to the word "tv"
model.wv.most_similar(positive=['tv'])

In [ ]:
# Or we may want to check the similarity between two words
model.wv.similarity("election", "vote")

In [ ]:
model.wv.similarity("election", "golf")

We may want to visualize word similarities as well! However, we are unable to visualize a 100-dimensional vector, hence we need to use dimensionality reduction algorithms. We will use t-SNE, which is a non-linear dimensionality reduction algorithm. Here we will use the **google-300 model** to show big differences. We will use the following helper function.

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import seaborn as sns
sns.set_style("darkgrid")


def tsnescatterplot(wv, word, list_names):
    """ Plot in seaborn the results from the t-SNE dimensionality reduction algorithm of the vectors of a query word,
    its list of most similar words, and a list of words.
    """
    arrays = np.empty((0, wv.vector_size), dtype='f')
    word_labels = [word]
    color_list = ['red']

    # adds the vector of the query word
    arrays = np.append(arrays, wv.__getitem__([word]), axis=0)

    # gets list of most similar words
    close_words = wv.most_similar([word])

    # adds the vector for each of the closest words to the array
    for wrd_score in close_words:
        wrd_vector = wv.__getitem__([wrd_score[0]])
        word_labels.append(wrd_score[0])
        color_list.append('blue')
        arrays = np.append(arrays, wrd_vector, axis=0)

    # adds the vector for each of the words from list_names to the array
    for wrd in list_names:
        wrd_vector = wv.__getitem__([wrd])
        word_labels.append(wrd)
        color_list.append('green')
        arrays = np.append(arrays, wrd_vector, axis=0)

    # Reduces the dimensionality from 300 to 50 dimensions with PCA
    reduc = PCA(n_components=20).fit_transform(arrays)

    # Finds t-SNE coordinates for 2 dimensions
    np.set_printoptions(suppress=True)

    Y = TSNE(n_components=2, random_state=0,
             perplexity=15).fit_transform(reduc)

    # Sets everything up to plot
    df = pd.DataFrame({'x': [x for x in Y[:, 0]],
                       'y': [y for y in Y[:, 1]],
                       'words': word_labels,
                       'color': color_list})

    fig, _ = plt.subplots()
    fig.set_size_inches(9, 9)

    # Basic plot
    p1 = sns.regplot(data=df,
                     x="x",
                     y="y",
                     fit_reg=False,
                     marker="o",
                     scatter_kws={'s': 40,
                                  'facecolors': df['color']
                                  }
                     )

    # Adds annotations one by one with a loop
    for line in range(0, df.shape[0]):
        p1.text(df["x"][line],
                df['y'][line],
                '  ' + df["words"][line].title(),
                horizontalalignment='left',
                verticalalignment='bottom', size='medium',
                color=df['color'][line],
                weight='normal'
                ).set_size(15)

    plt.xlim(Y[:, 0].min()-50, Y[:, 0].max()+50)
    plt.ylim(Y[:, 1].min()-50, Y[:, 1].max()+50)

    plt.title('t-SNE visualization for {}'.format(word.title()))


In [ ]:
# 10 words most similar to "parliament" and 10 words most similar to "football"
tsnescatterplot(google_wv, 'parliament', [x[0] for x in google_wv.most_similar(positive=["football"])])

Since "football" and "parliament" are from two different words, the embedding of their closest words can be easily separated (points are visibly grouped).

In [ ]:
# 10 words most similar to "spaceship" and 10 words most similar to "spacecraft"
tsnescatterplot(google_wv, 'spaceship', [x[0] for x in google_wv.most_similar(positive=["spacecraft"])])

However, in this case, the "spaceship" is only a different name for the "spacecraft", so their embeddings should also look similar and similar words are mixed.

#### Similarity between documents

Now, to measure the similarity between two documents, we can represent them as weighted sums of word embeddings:

In [ ]:
d1 = bbc_data['text'].iloc[1]  # bussiness article
d2 = bbc_data['text'].iloc[2]  # sport article
d3 = bbc_data['text'].iloc[3]  # another sport article

bbc_data[['text', 'category']].iloc[1:4]

In [ ]:
def document_weighted_sum(document, model, bigrams):
    cleaned_doc = preprocess_doc(document)
    doc_bigrams = bigrams[cleaned_doc]

    vec_count = 0
    # Empty vector containing only 0s
    sum_v = np.zeros((model.wv.vector_size))

    if len(sum_v) == 0:
        return sum_v

    # add each word embedding vector to the sum vector
    for token in doc_bigrams:
        if token in model.wv.key_to_index:
            sum_v = np.add(sum_v, model.wv.get_vector(token))
            vec_count += 1
    
    # divide by the number of added vectors
    sum_v = np.divide(sum_v, vec_count)

    return sum_v

Let's check the similartiy vector between all three documents:

In [ ]:
weighted_vector1 = document_weighted_sum(d1, model, bigrams)
weighted_vector2 = document_weighted_sum(d2, model, bigrams)
weighted_vector3 = document_weighted_sum(d3, model, bigrams)

In [ ]:
vector_cos(weighted_vector1, weighted_vector2)

In [ ]:
vector_cos(weighted_vector1, weighted_vector3)

In [ ]:
vector_cos(weighted_vector2, weighted_vector3)

As you can see, the model clearly shows that the biggest similarity is between articles about sport.

### 11.6. Similarity measurement - word embeddings and one-hot vectors

Let's see why cosine similarity works well with word embeddings and doesn't work with one-hot vectors. Let's say we have two similar sentences in terms of semantics:
- Since I enjoy spending time with animals, I decided to buy a dog
- You are thinking of adopting a cat because you like pets


We expect these two sentences to have relatively high similarity measures since they both are about enjoying animals and wanting to get one. We will use pre-trained word embeddings from the google news model.

In [ ]:
sentences = ["Since I enjoy spending time with animals I decided to buy a dog", "You are thinking of adopting a cat because you like pets"]
split_sentences = [word_tokenize(doc) for doc in sentences]
print(split_sentences)

Let's create a `sentence_weighted_sum` function to extract the meaning of each sentence. We will not use bigrams in this case - just simple words.

In [ ]:
def sentence_weighted_sum(sentence, wv):
    doc_tokenized = preprocess_doc(sentence)
    sum_v = np.zeros((wv.vector_size))

    if len(sum_v) == 0:
        return sum_v

    for token in doc_tokenized:
        sum_v = np.add(sum_v, wv.get_vector(token))
    
    sum_v = np.divide(sum_v, wv.vector_size)

    return sum_v

In [ ]:
s1_meaning = sentence_weighted_sum(sentences[0], google_wv)
s2_meaning = sentence_weighted_sum(sentences[1], google_wv)

In [ ]:
vector_cos(s1_meaning, s2_meaning)

As you can see, the cosine similarity for these two sentences is high when using word embeddings. How about one-hot vectors?

Firstly, let's process both sentences in a similar way to what we did for word embeddings (tokenize, remove stopwords).

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(lowercase=False, token_pattern=r"\w+")
processed_s1 = preprocess_doc(sentences[0])
processed_s2 = preprocess_doc(sentences[1])

transformed = count_vectorizer.fit_transform([" ".join(processed_s1), " ".join(processed_s2)])

Let's the word-features and transformed sentences.

In [ ]:
# Let's inspect our features
print(count_vectorizer.get_feature_names())
print(transformed[0].toarray())
print(transformed[1].toarray())

Now, let's calculate the cosine similarity between these two vectors!

In [ ]:
vector_cos(transformed[0].toarray(), transformed[1].toarray().T)

As you can see, the cosine similarity for one-hot vectors is 0. This is because even these sentences have similar meanings, they do not have any common words (after stopwords removal)! Since bag-of-words is based on word counts, if two documents have no common words, they are totally different (from the BOW perspective). However, word embeddings capture word similarities ("dog" is close to "cat", "buy" is close to "adopt", etc.) and when applied, they can show the similarity in this case.